## 制作yuanfazhao数据的肺部Mask


采用形态学中的阈值分析法



方法不是很好，可能只有左肺或右肺，或者肿瘤部位直接被当作背景了。

### 存放原始数据中每个病人的CT/PET序列总数
patient_ct_pet_len = {"wangyunqing":205,"tangliru":287,"ruanguanfu":205,"wulong":239,"wulingbo":287,"wangguiping":287,"liwanfeng":239,
"jinyanlai":287,"huangxueqi":287,"xiafafu":287,"liushaogang":239,"lixiuqing":205,"wangwuping":287,"wanglijuan":205,"wanhongliang":239}


### 每个序列倒序排列后的序号
列出的序号都是肺部以外的人体，分析时去掉，手动找到，得想个其他方法解决

"wangyunqing":1-72，147-205

"tangliru":1-135，205-287

"ruanguanfu":1-84，148-205

"wulong":1-111,179-239

"wulingbo":1-127，208-287

"wangguiping":1-135，210-287

"liwanfeng":1-98，175-239

"jinyanlai":1-123，202-287

"huangxueqi":1-131，209-287

"xiafafu":1-120，197-287

"liushaogang":1-112，169-239

"lixiuqing":1-87，148-205

"wangwuping":1-141，207-287

"wanglijuan":1-88，152-205

"wanhongliang":1-115，177-239

In [1]:
import numpy as np 
import pydicom
import os
import scipy.ndimage
import matplotlib.pyplot as plt

from skimage import measure, morphology
import time
import pdb 

In [2]:
# 原始数据中每个病人的CT/PET序列总数
patient_ct_pet_len = {"wangyunqing":205,"tangliru":287,"ruanguanfu":205,"wulong":239,"wulingbo":287,"wangguiping":287,"liwanfeng":239, "jinyanlai":287,"huangxueqi":287,"xiafafu":287,"liushaogang":239,"lixiuqing":205,"wangwuping":287,"wanglijuan":205,"wanhongliang":239}

In [3]:
# ITKsnap中需要切除的序号
patient_name = list(patient_ct_pet_len.keys())
print(patient_name)

# 存放切除序号的字典
num_dic = {}
for name in patient_name:
    if name=="wangyunqing":
        l1 = list(range(1,73))
        l1.extend(range(147,206))
        num_dic[name] = l1
    elif name=="tangliru":
        l1 = list(range(1,136))
        l1.extend(range(205,288))
        num_dic[name] = l1
    elif name == "ruanguanfu":
        l1 = list(range(1,85))
        l1.extend(range(148,206))
        num_dic[name] = l1
    elif name == "wulong":
        l1 = list(range(1,112))
        l1.extend(range(179,240))
        num_dic[name] = l1
    elif name == "wulingbo":
        l1 = list(range(1,128))
        l1.extend(range(208,288))
        num_dic[name] = l1
    elif name == "wangguiping":
        l1 = list(range(1,136))
        l1.extend(range(210,288))
        num_dic[name] = l1
    elif name =="liwanfeng":
        l1 = list(range(1,99))
        l1.extend(range(175,240))
        num_dic[name] = l1
    elif name =="jinyanlai":
        l1 = list(range(1,124))
        l1.extend(range(202,288))
        num_dic[name] = l1
    elif name=="huangxueqi":
        l1 = list(range(1,132))
        l1.extend(range(209,288))
        num_dic[name] = l1
    elif name=="xiafafu":
        l1 = list(range(1,121))
        l1.extend(range(197,288))
        num_dic[name] = l1
    elif name=="liushaogang":
        l1 = list(range(1,113))
        l1.extend(range(169,240))
        num_dic[name] = l1
    elif name=="lixiuqing":
        l1 = list(range(1,88))
        l1.extend(range(148,206))
        num_dic[name] = l1
    elif name=="wangwuping":
        l1 = list(range(1,142))
        l1.extend(range(207,288))
        num_dic[name] = l1
    elif name=="wanglijuan":
        l1 = list(range(1,89))
        l1.extend(range(152,206))
        num_dic[name] = l1
    elif name=="wanhongliang":
        l1 = list(range(1,116))
        l1.extend(range(177,240))
        num_dic[name] = l1
    else:
        print("Error")

['wangyunqing', 'tangliru', 'ruanguanfu', 'wulong', 'wulingbo', 'wangguiping', 'liwanfeng', 'jinyanlai', 'huangxueqi', 'xiafafu', 'liushaogang', 'lixiuqing', 'wangwuping', 'wanglijuan', 'wanhongliang']


In [4]:
print(num_dic["wangyunqing"])
print(num_dic["wangguiping"])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 10

In [5]:
# 对应的文件名
def make_filename(num_dic,patient_ct_pet_len):
    filename_dic = {}
    
    for k,v in num_dic.items():
        num = patient_ct_pet_len[k] + 1
        v_len = len(v)
        
        for i in range(v_len):
            v[i] = num-v[i]
            le = len(str(v[i]))
            
            if(le==1):
                v[i] = "00"+str(v[i])
            elif(le==2):
                v[i] = "0"+str(v[i])
            elif(le==3):
                v[i] = str(v[i])
            else:
                print("Error")
                
        filename_dic[k] = v
        
    return filename_dic

In [6]:
filename = make_filename(num_dic,patient_ct_pet_len)

In [7]:
print(filename["wangyunqing"])

['205', '204', '203', '202', '201', '200', '199', '198', '197', '196', '195', '194', '193', '192', '191', '190', '189', '188', '187', '186', '185', '184', '183', '182', '181', '180', '179', '178', '177', '176', '175', '174', '173', '172', '171', '170', '169', '168', '167', '166', '165', '164', '163', '162', '161', '160', '159', '158', '157', '156', '155', '154', '153', '152', '151', '150', '149', '148', '147', '146', '145', '144', '143', '142', '141', '140', '139', '138', '137', '136', '135', '134', '059', '058', '057', '056', '055', '054', '053', '052', '051', '050', '049', '048', '047', '046', '045', '044', '043', '042', '041', '040', '039', '038', '037', '036', '035', '034', '033', '032', '031', '030', '029', '028', '027', '026', '025', '024', '023', '022', '021', '020', '019', '018', '017', '016', '015', '014', '013', '012', '011', '010', '009', '008', '007', '006', '005', '004', '003', '002', '001']


In [9]:
# 删除文件
data_path = "F:\\data\\yuanfazhao\\yuanfazhao_data_07"

patient_path = [os.path.join(data_path,s) for s in os.listdir(data_path)]
print(patient_path)

patient_ct_path = []
for v in patient_path:
    t = os.listdir(v)
    s = os.path.join(v,t[0])
    patient_ct_path.append(s)
    
print(patient_ct_path)

['F:\\data\\yuanfazhao_data_07\\N0-1978-wangyunqing', 'F:\\data\\yuanfazhao_data_07\\N0-2115-tangliru', 'F:\\data\\yuanfazhao_data_07\\N0-2201-ruanguanfu', 'F:\\data\\yuanfazhao_data_07\\N0-484-wulong', 'F:\\data\\yuanfazhao_data_07\\N0-485-wulingbo', 'F:\\data\\yuanfazhao_data_07\\N0-60-wangguiping', 'F:\\data\\yuanfazhao_data_07\\N1-2525-liwanfeng', 'F:\\data\\yuanfazhao_data_07\\N1-2621-jinyanlai', 'F:\\data\\yuanfazhao_data_07\\N2-1062-huangxueqi', 'F:\\data\\yuanfazhao_data_07\\N2-1901-xiafafu', 'F:\\data\\yuanfazhao_data_07\\N2-2403-liushaogang', 'F:\\data\\yuanfazhao_data_07\\N2-2506-lixiuqing', 'F:\\data\\yuanfazhao_data_07\\N2-541-wangwuping', 'F:\\data\\yuanfazhao_data_07\\N2-557-wanglijuan', 'F:\\data\\yuanfazhao_data_07\\N2-607-wanhongliang']
['F:\\data\\yuanfazhao_data_07\\N0-1978-wangyunqing\\C4NEH5E20', 'F:\\data\\yuanfazhao_data_07\\N0-2115-tangliru\\CZPJUFO1Y', 'F:\\data\\yuanfazhao_data_07\\N0-2201-ruanguanfu\\CYPYNZS0T', 'F:\\data\\yuanfazhao_data_07\\N0-484-wulong\\

In [10]:
for v in patient_ct_path:
    # tt = os.listdir(v)
    # src_name = os.path.join(v,t for t in tt)
    patient_name = v.split("\\")[3].split("-")[-1]
    file_name = filename[patient_name]
    for f in file_name:
        new_name = os.path.join(v,f)+".dcm"
        if(os.path.exists(new_name)):
            os.remove(new_name)

### 下边提取肺部MASK

In [19]:
# 载入图像
def load_scan(path):
    # 读入slices存储的是一个序列多个dcm内容的列表
    slices = [pydicom.dcmread(path + '\\' + s) for s in os.listdir(path)]
    # 排序
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
        
    return slices

In [12]:
# 将pixel转为HU
def get_pixels_hu(slices):
    # 2维切片堆叠为3维scan。（矩阵）
    image = np.stack([s.pixel_array for s in slices])

    # 转换为int16类型，有时就是int16类型
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # CT扫描边界之外的灰度值固定为-2000，将这些值设定为0
    image[image==-2000] = 0
    
    # Convert to Hounsfield units (HU)
    for slice_number in range(len(slices)):        
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(np.float64)
            image[slice_number] = image[slice_number].astype(np.int16)
            
        image[slice_number] += np.int16(intercept)
    
    # 返回image(int16),像素间距（z,x,y)
    return np.array(image, dtype=np.int16), np.array([slices[0].SliceThickness] + list(slices[0].PixelSpacing), dtype=np.float32)

In [13]:
# 单独对每张图二值化
def binarize_per_slice(image, spacing, intensity_th=-600, sigma=1, area_th=30, eccen_th=0.99, bg_patch_size=10):
    # bw存储的就是二值True或False
    bw = np.zeros(image.shape, dtype=bool)
    
    # 准备一个mask.corner的值为nan。也就是正方形中有个内接圆，值为1，其余为nan
    # prepare a mask, with all corner values set to nan
    image_size = image.shape[1]
    grid_axis = np.linspace(-image_size/2+0.5, image_size/2-0.5, image_size)
    x, y = np.meshgrid(grid_axis, grid_axis)
    d = (x**2+y**2)**0.5
    nan_mask = (d<image_size/2).astype(float)
    nan_mask[nan_mask == 0] = np.nan

    # bw开始赋值
    for i in range(image.shape[0]):
        # 检查图像的角像素是否相同，如果相同，则在高斯滤波前与nan_mask相乘
        if len(np.unique(image[i, 0:bg_patch_size, 0:bg_patch_size])) == 1:
            # current_bw为一个二值（与阈值比较后的）
            current_bw = scipy.ndimage.filters.gaussian_filter(np.multiply(image[i].astype('float32'), nan_mask), sigma, truncate=2.0) < intensity_th
        else:
            current_bw = scipy.ndimage.filters.gaussian_filter(image[i].astype('float32'), sigma, truncate=2.0) < intensity_th
        
        # 选择连通区域
        # label函数返回二值图中的连通区的label矩阵，每一个连通区由一个值定义
        label = measure.label(current_bw)
        # regionprops()获取label中每一个连通区的属性。
        properties = measure.regionprops(label)
        # valid_lable是一个有效label值集合
        valid_label = set()
        for prop in properties:
            if prop.area * spacing[1] * spacing[2] > area_th and prop.eccentricity < eccen_th:
                valid_label.add(prop.label)
        # current_bw为符合条件的二值图，利用np.in1d()函数。
        current_bw = np.in1d(label, list(valid_label)).reshape(label.shape)
        bw[i] = current_bw
        
    return bw

In [14]:
# 所有切片一起分析
def all_slice_analysis(bw, spacing, cut_num=0, vol_limit=[0.68, 8.2], area_th=6e3, dist_th=62):
    # 在某些情况下，首先需要删除几个顶层
    # in some cases, several top layers need to be removed first
    if cut_num > 0:
        bw0 = np.copy(bw)
        # bw[-cut_num]后cut_num张
        bw[-cut_num:] = False
    # bw是3维的
    label = measure.label(bw, connectivity=1)
    # remove components access to corners
    mid = int(label.shape[2] / 2)
    # bg_label为以下几个位置的值，set用于去掉重复值
    bg_label = set([label[0, 0, 0], label[0, 0, -1], label[0, -1, 0], label[0, -1, -1], \
                    label[-1-cut_num, 0, 0], label[-1-cut_num, 0, -1], label[-1-cut_num, -1, 0], label[-1-cut_num, -1, -1], \
                    label[0, 0, mid], label[0, -1, mid], label[-1-cut_num, 0, mid], label[-1-cut_num, -1, mid]])
    # label的值已将背景设为0了。
    for l in bg_label:
        label[label == l] = 0
        
    # select components based on volume
    # 通过volume寻找连通区
    properties = measure.regionprops(label)
    for prop in properties:
        if prop.area * spacing.prod() < vol_limit[0] * 1e6 or prop.area * spacing.prod() > vol_limit[1] * 1e6:
            label[label == prop.label] = 0
            
    # prepare a distance map for further analysis
    # 准备一个distance map用作未来的分析
    x_axis = np.linspace(-label.shape[1]/2+0.5, label.shape[1]/2-0.5, label.shape[1]) * spacing[1]
    y_axis = np.linspace(-label.shape[2]/2+0.5, label.shape[2]/2-0.5, label.shape[2]) * spacing[2]
    x, y = np.meshgrid(x_axis, y_axis)
    d = (x**2+y**2)**0.5

    vols = measure.regionprops(label)
    valid_label = set()
    # select components based on their area and distance to center axis on all slices
    for vol in vols:
        single_vol = label == vol.label
        slice_area = np.zeros(label.shape[0])
        min_distance = np.zeros(label.shape[0])
        for i in range(label.shape[0]):
            slice_area[i] = np.sum(single_vol[i]) * np.prod(spacing[1:3])
            min_distance[i] = np.min(single_vol[i] * d + (1 - single_vol[i]) * np.max(d))
        
        if np.average([min_distance[i] for i in range(label.shape[0]) if slice_area[i] > area_th]) < dist_th:
            valid_label.add(vol.label)
    
    # 3维图像中符合条件的二值图。        
    bw = np.in1d(label, list(valid_label)).reshape(label.shape)
    
    # fill back the parts removed earlier
    if cut_num > 0:
        # bw1 is bw with removed slices, bw2 is a dilated version of bw, part of their intersection is returned as final mask
        bw1 = np.copy(bw)
        bw1[-cut_num:] = bw0[-cut_num:]
        bw2 = np.copy(bw)
        bw2 = scipy.ndimage.binary_dilation(bw2, iterations=cut_num)
        bw3 = bw1 & bw2
        label = measure.label(bw, connectivity=1)
        label3 = measure.label(bw3, connectivity=1)
        l_list = list(set(np.unique(label)) - {0})
        valid_l3 = set()
        for l in l_list:
            indices = np.nonzero(label==l)
            l3 = label3[indices[0][0], indices[1][0], indices[2][0]]
            if l3 > 0:
                valid_l3.add(l3)
        bw = np.in1d(label3, list(valid_l3)).reshape(label3.shape)
    
    # 返回bw,以及其中包含的label值的数量。
    return bw, len(valid_label)

In [15]:
# 孔洞填充
def fill_hole(bw):
    # fill 3d holes
    label = measure.label(~bw)
    # idendify corner components
    bg_label = set([label[0, 0, 0], label[0, 0, -1], label[0, -1, 0], label[0, -1, -1], \
                    label[-1, 0, 0], label[-1, 0, -1], label[-1, -1, 0], label[-1, -1, -1]])
    bw = ~np.in1d(label, list(bg_label)).reshape(label.shape)
    
    return bw

In [16]:
# 两个肺部的掩码
def two_lung_only(bw, spacing, max_iter=22, max_ratio=4.8):    
    def extract_main(bw, cover=0.95):
        for i in range(bw.shape[0]):
            current_slice = bw[i]
            label = measure.label(current_slice)
            properties = measure.regionprops(label)
            properties.sort(key=lambda x: x.area, reverse=True)
            area = [prop.area for prop in properties]
            count = 0
            sum = 0
            while sum < np.sum(area)*cover:
                sum = sum+area[count]
                count = count+1
            filter = np.zeros(current_slice.shape, dtype=bool)
            for j in range(count):
                bb = properties[j].bbox
                filter[bb[0]:bb[2], bb[1]:bb[3]] = filter[bb[0]:bb[2], bb[1]:bb[3]] | properties[j].convex_image
            bw[i] = bw[i] & filter
           
        label = measure.label(bw)
        properties = measure.regionprops(label)
        properties.sort(key=lambda x: x.area, reverse=True)
        bw = label==properties[0].label

        return bw
    
    def fill_2d_hole(bw):
        for i in range(bw.shape[0]):
            current_slice = bw[i]
            label = measure.label(current_slice)
            properties = measure.regionprops(label)
            for prop in properties:
                bb = prop.bbox
                current_slice[bb[0]:bb[2], bb[1]:bb[3]] = current_slice[bb[0]:bb[2], bb[1]:bb[3]] | prop.filled_image
            bw[i] = current_slice

        return bw
    
    # 进行腐蚀
    found_flag = False
    iter_count = 0
    bw0 = np.copy(bw)
    while not found_flag and iter_count < max_iter:
        label = measure.label(bw, connectivity=2)
        properties = measure.regionprops(label)
        properties.sort(key=lambda x: x.area, reverse=True)
        if len(properties) > 1 and properties[0].area/properties[1].area < max_ratio:
            found_flag = True
            bw1 = label == properties[0].label
            bw2 = label == properties[1].label
        else:
            # binary_erosion是腐蚀运算
            bw = scipy.ndimage.binary_erosion(bw)
            iter_count = iter_count + 1
    
    if found_flag:
        # morphology.distance_transform_edt()求距离
        d1 = scipy.ndimage.morphology.distance_transform_edt(bw1 == False, sampling=spacing)
        d2 = scipy.ndimage.morphology.distance_transform_edt(bw2 == False, sampling=spacing)
        bw1 = bw0 & (d1 < d2)
        bw2 = bw0 & (d1 > d2)
                
        bw1 = extract_main(bw1) # 左肺
        bw2 = extract_main(bw2) # 右肺
        
    else:
        bw1 = bw0
        bw2 = np.zeros(bw.shape).astype('bool')
        
    bw1 = fill_2d_hole(bw1) 
    bw2 = fill_2d_hole(bw2)
    bw = bw1 | bw2

    return bw1, bw2, bw

In [20]:
def make_mask(case_path):
    
    case = load_scan(case_path)
    
    case_pixels, spacing = get_pixels_hu(case)
    
    bw = binarize_per_slice(case_pixels, spacing)
    
    flag = 0
    cut_num = 0
    cut_step = 2
    bw0 = np.copy(bw)
    while flag == 0 and cut_num < bw.shape[0]:
        bw = np.copy(bw0)

        bw, flag = all_slice_analysis(bw, spacing, cut_num=cut_num, vol_limit=[0.68,7.5])
        cut_num = cut_num + cut_step
    
    bw = fill_hole(bw)
    
    bw1, bw2, bw = two_lung_only(bw, spacing)

    return case_pixels,bw,spacing

In [42]:
from PIL import Image


# 以下代码是用来保存的时候保存为二值图的表
threshold = 1
table = []
for i in range(256):
    if i < threshold:
        table.append(0)
    else:
        table.append(1)

### 下面是一个一个病人的提取的肺部Mask
因为循环运行会宕机

In [43]:
# 获取第一个病人的掩码
i = 0
ct_path = patient_ct_path[0]
print(ct_path)
pixels,bw,spacing = make_mask(ct_path)
# 保存为BMP图像
bw = bw[::-1]
bw = bw.astype(np.uint8)
m = 60
for n in range(bw.shape[0]):
    k = "%03d" % m
    new_filename = str(k)
    img = Image.fromarray(bw[n])
    img=img.point(table,"1")
    img.save("{}\\{}.bmp".format(ct_path,new_filename))
    m = m+1

F:\data\yuanfazhao_data_07\N0-1978-wangyunqing\C4NEH5E20


D:\Soft\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


In [44]:
# 获取第2个病人的掩码
i = 1
ct_path = patient_ct_path[1]
print(ct_path)
pixels,bw,spacing = make_mask(ct_path)
# 保存为BMP图像
bw = bw[::-1]
bw = bw.astype(np.uint8)
m = 84
for n in range(bw.shape[0]):
    k = "%03d" % m
    new_filename = str(k)
    img = Image.fromarray(bw[n])
    img=img.point(table,"1")
    img.save("{}\\{}.bmp".format(ct_path,new_filename))
    m = m+1

F:\data\yuanfazhao_data_07\N0-2115-tangliru\CZPJUFO1Y


D:\Soft\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


In [45]:
# 获取第3个病人的掩码
i = 2
ct_path = patient_ct_path[2]
print(ct_path)
pixels,bw,spacing = make_mask(ct_path)
# 保存为BMP图像
bw = bw[::-1]
bw = bw.astype(np.uint8)
m = 59
for n in range(bw.shape[0]):
    k = "%03d" % m
    new_filename = str(k)
    img = Image.fromarray(bw[n])
    img=img.point(table,"1")
    img.save("{}\\{}.bmp".format(ct_path,new_filename))
    m = m+1

F:\data\yuanfazhao_data_07\N0-2201-ruanguanfu\CYPYNZS0T


D:\Soft\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


In [46]:
# 获取第4个病人的掩码
i = 3
ct_path = patient_ct_path[3]
print(ct_path)
pixels,bw,spacing = make_mask(ct_path)
# 保存为BMP图像
bw = bw[::-1]
bw = bw.astype(np.uint8)
m = 62
for n in range(bw.shape[0]):
    k = "%03d" % m
    new_filename = str(k)
    img = Image.fromarray(bw[n])
    img=img.point(table,"1")
    img.save("{}\\{}.bmp".format(ct_path,new_filename))
    m = m+1

F:\data\yuanfazhao_data_07\N0-484-wulong\CE1VOZRHT


D:\Soft\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


In [47]:
# 获取第5个病人的掩码
i = 4
ct_path = patient_ct_path[4]
print(ct_path)
pixels,bw,spacing = make_mask(ct_path)
# 保存为BMP图像
bw = bw[::-1]
bw = bw.astype(np.uint8)
m = 81
for n in range(bw.shape[0]):
    k = "%03d" % m
    new_filename = str(k)
    img = Image.fromarray(bw[n])
    img=img.point(table,"1")
    img.save("{}\\{}.bmp".format(ct_path,new_filename))
    m = m+1

F:\data\yuanfazhao_data_07\N0-485-wulingbo\CZXMCE2E0


D:\Soft\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


In [48]:
# 获取第6个病人的掩码
i = 5
ct_path = patient_ct_path[5]
print(ct_path)
pixels,bw,spacing = make_mask(ct_path)
# 保存为BMP图像
bw = bw[::-1]
bw = bw.astype(np.uint8)
m = 79
for n in range(bw.shape[0]):
    k = "%03d" % m
    new_filename = str(k)
    img = Image.fromarray(bw[n])
    img=img.point(table,"1")
    img.save("{}\\{}.bmp".format(ct_path,new_filename))
    m = m+1

F:\data\yuanfazhao_data_07\N0-60-wangguiping\C5YGD35LZ


D:\Soft\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


In [49]:
# 获取第7个病人的掩码
i = 6
ct_path = patient_ct_path[6]
print(ct_path)
pixels,bw,spacing = make_mask(ct_path)
# 保存为BMP图像
bw = bw[::-1]
bw = bw.astype(np.uint8)
m = 66
for n in range(bw.shape[0]):
    k = "%03d" % m
    new_filename = str(k)
    img = Image.fromarray(bw[n])
    img=img.point(table,"1")
    img.save("{}\\{}.bmp".format(ct_path,new_filename))
    m = m+1

F:\data\yuanfazhao_data_07\N1-2525-liwanfeng\CWJREMFFT


D:\Soft\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


In [50]:
# 获取第8个病人的掩码
i = 7
ct_path = patient_ct_path[7]
print(ct_path)
pixels,bw,spacing = make_mask(ct_path)
# 保存为BMP图像
bw = bw[::-1]
bw = bw.astype(np.uint8)
m = 87
for n in range(bw.shape[0]):
    k = "%03d" % m
    new_filename = str(k)
    img = Image.fromarray(bw[n])
    img=img.point(table,"1")
    img.save("{}\\{}.bmp".format(ct_path,new_filename))
    m = m+1

F:\data\yuanfazhao_data_07\N1-2621-jinyanlai\CWAYLK3IB


D:\Soft\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


In [51]:
# 获取第9个病人的掩码
i = 8
ct_path = patient_ct_path[8]
print(ct_path)
pixels,bw,spacing = make_mask(ct_path)
# 保存为BMP图像
bw = bw[::-1]
bw = bw.astype(np.uint8)
m = 80
for n in range(bw.shape[0]):
    k = "%03d" % m
    new_filename = str(k)
    img = Image.fromarray(bw[n])
    img=img.point(table,"1")
    img.save("{}\\{}.bmp".format(ct_path,new_filename))
    m = m+1

F:\data\yuanfazhao_data_07\N2-1062-huangxueqi\CWHIB2QD5


D:\Soft\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


In [52]:
# 获取第10个病人的掩码
i = 9
ct_path = patient_ct_path[9]
print(ct_path)
pixels,bw,spacing = make_mask(ct_path)
# 保存为BMP图像
bw = bw[::-1]
bw = bw.astype(np.uint8)
m = 92
for n in range(bw.shape[0]):
    k = "%03d" % m
    new_filename = str(k)
    img = Image.fromarray(bw[n])
    img=img.point(table,"1")
    img.save("{}\\{}.bmp".format(ct_path,new_filename))
    m = m+1

F:\data\yuanfazhao_data_07\N2-1901-xiafafu\CT2XWRBHT


D:\Soft\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


In [53]:
# 获取第11个病人的掩码
i = 10
ct_path = patient_ct_path[10]
print(ct_path)
pixels,bw,spacing = make_mask(ct_path)
# 保存为BMP图像
bw = bw[::-1]
bw = bw.astype(np.uint8)
m = 72
for n in range(bw.shape[0]):
    k = "%03d" % m
    new_filename = str(k)
    img = Image.fromarray(bw[n])
    img=img.point(table,"1")
    img.save("{}\\{}.bmp".format(ct_path,new_filename))
    m = m+1

F:\data\yuanfazhao_data_07\N2-2403-liushaogang\CNMDHQGZE


D:\Soft\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


In [54]:
# 获取第12个病人的掩码
i = 11
ct_path = patient_ct_path[11]
print(ct_path)
pixels,bw,spacing = make_mask(ct_path)
# 保存为BMP图像
bw = bw[::-1]
bw = bw.astype(np.uint8)
m = 59
for n in range(bw.shape[0]):
    k = "%03d" % m
    new_filename = str(k)
    img = Image.fromarray(bw[n])
    img=img.point(table,"1")
    img.save("{}\\{}.bmp".format(ct_path,new_filename))
    m = m+1

F:\data\yuanfazhao_data_07\N2-2506-lixiuqing\CZ5TGIPGI


D:\Soft\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


In [55]:
# 获取第13个病人的掩码
i = 12
ct_path = patient_ct_path[12]
print(ct_path)
pixels,bw,spacing = make_mask(ct_path)
# 保存为BMP图像
bw = bw[::-1]
bw = bw.astype(np.uint8)
m = 82
for n in range(bw.shape[0]):
    k = "%03d" % m
    new_filename = str(k)
    img = Image.fromarray(bw[n])
    img=img.point(table,"1")
    img.save("{}\\{}.bmp".format(ct_path,new_filename))
    m = m+1

F:\data\yuanfazhao_data_07\N2-541-wangwuping\C1UQCBBUK


D:\Soft\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


In [56]:
# 获取第14个病人的掩码
i = 13
ct_path = patient_ct_path[13]
print(ct_path)
pixels,bw,spacing = make_mask(ct_path)
# 保存为BMP图像
bw = bw[::-1]
bw = bw.astype(np.uint8)
m = 55
for n in range(bw.shape[0]):
    k = "%03d" % m
    new_filename = str(k)
    img = Image.fromarray(bw[n])
    img=img.point(table,"1")
    img.save("{}\\{}.bmp".format(ct_path,new_filename))
    m = m+1

F:\data\yuanfazhao_data_07\N2-557-wanglijuan\CJPWWVPSY


D:\Soft\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


In [57]:
# 获取第15个病人的掩码
i = 14
ct_path = patient_ct_path[14]
print(ct_path)
pixels,bw,spacing = make_mask(ct_path)
# 保存为BMP图像
bw = bw[::-1]
bw = bw.astype(np.uint8)
m = 64
for n in range(bw.shape[0]):
    k = "%03d" % m
    new_filename = str(k)
    img = Image.fromarray(bw[n])
    img=img.point(table,"1")
    img.save("{}\\{}.bmp".format(ct_path,new_filename))
    m = m+1

F:\data\yuanfazhao_data_07\N2-607-wanhongliang\CMGT0224F


D:\Soft\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less
